In [1]:
import os
import pandas as pd

In [66]:
business = pd.read_csv(os.getcwd()+'/data/Businesses.csv')

In [23]:
income = pd.read_csv(os.getcwd()+'/data/Income.csv')

In [24]:
polling = pd.read_csv(os.getcwd()+'/data/PollingPlaces2019.csv')

In [25]:
population = pd.read_csv(os.getcwd()+'/data/Population.csv')

In [26]:
stops = pd.read_csv(os.getcwd()+'/data/Stops.txt')


# Data Cleaning and Exploration

Below is an exploration of the imported data sets for features and imperfections

In [67]:
business.shape

(12217, 11)

In [75]:
business

,industry_code,industry_name,sa2_code,sa2_name,0_to_50k_businesses,50k_to_200k_businesses,200k_to_2m_businesses,2m_to_5m_businesses,5m_to_10m_businesses,10m_or_more_businesses,total_businesses
0,A,"Agriculture, Forestry and Fishing",101021007,Braidwood,136,92,63,4,0,0,296
1,A,"Agriculture, Forestry and Fishing",101021008,Karabar,6,3,0,0,0,0,9
2,A,"Agriculture, Forestry and Fishing",101021009,Queanbeyan,6,4,3,0,0,3,15
3,A,"Agriculture, Forestry and Fishing",101021010,Queanbeyan - East,0,3,0,0,0,0,3
4,A,"Agriculture, Forestry and Fishing",101021012,Queanbeyan West - Jerrabomberra,7,4,5,0,0,0,16
...,...,...,...,...,...,...,...,...,...,...,...
12212,S,Other Services,128021538,Sutherland - Kirrawee,21,66,58,3,3,0,152
12213,S,Other Services,128021607,Engadine,13,41,31,3,0,0,87
12214,S,Other Services,128021608,Loftus - Yarrawarrah,0,10,10,0,0,0,22
12215,S,Other Services,128021609,Woronora Heights,0,3,5,0,0,0,9


In [65]:
income.shape

(642, 6)

In [74]:
income

,sa2_code21,sa2_name,earners,median_age,median_income,mean_income
0,101021007,Braidwood,2467,51,46640,68904
1,101021008,Karabar,5103,42,65564,69672
2,101021009,Queanbeyan,7028,39,63528,69174
3,101021010,Queanbeyan - East,3398,39,66148,74162
4,101021012,Queanbeyan West - Jerrabomberra,8422,44,78630,91981
...,...,...,...,...,...,...
637,128021537,Royal National Park,14,37,36980,47584
638,128021538,Sutherland - Kirrawee,13895,41,64940,74867
639,128021607,Engadine,10239,43,63695,72995
640,128021608,Loftus - Yarrawarrah,4424,45,63087,76440


In [62]:
polling.shape

(2930, 17)

In [63]:
population.shape

(373, 21)

In [64]:
stops.shape

(114718, 9)

In [40]:
nan_count = stops[specific_column].isna().sum()

print(stops.iloc[50000])

stop_id                                   G232163
stop_code                                     NaN
stop_name              Morpeth Rd Near Raworth Av
stop_lat                               -32.726616
stop_lon                               151.613898
location_type                                 1.0
parent_station                                NaN
wheelchair_boarding                             0
platform_code                                 NaN
Name: 50000, dtype: object
